In [ ]:
%load_ext jupyter_black

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Data preparation
We first need to prepare the data set: download the data, split train/val set and do normalisation. This part is taken from https://teaching.pages.centralesupelec.fr/deeplearning-lectures-build/00-pytorch-fashionMnist.html

In [ ]:
# 1. Get the data
# Change path accordingly
dataset_dir = "archive"
train_valid_dataset = torchvision.datasets.FashionMNIST(
    root=dataset_dir, train=True, transform=None, download=True
)
train_dataset, valid_dataset = torch.utils.data.dataset.random_split(
    train_valid_dataset, [0.8, 0.2]
)
test_dataset = torchvision.datasets.FashionMNIST(
    root=dataset_dir, transform=None, train=False  # transforms.ToTensor(),
)

In [ ]:
# 2. Compute the normalisation
ToTensor = transforms.ToTensor()  # Transform pil image to pytorch Tensor
mean_ = torch.zeros_like(ToTensor(train_dataset[0][0]))
std_ = torch.zeros_like(ToTensor(train_dataset[0][0]))
for img, _ in train_dataset:
    mean_ += ToTensor(img)
mean_ /= len(train_dataset)
for img, _ in train_dataset:
    std_ += (mean_ - ToTensor(img)) ** 2
std_ /= len(train_dataset)
std_ = torch.sqrt(std_)
std_[std_ == 0] = 1

In [ ]:
# 3. Prepare pytorch dataset
class DatasetTransformer(torch.utils.data.Dataset):
    def __init__(self, base_dataset, transform):
        self.base_dataset = base_dataset
        self.transform = transform

    def __getitem__(self, index):
        img, target = self.base_dataset[index]
        return self.transform(img), target

    def __len__(self):
        return len(self.base_dataset)


train_data_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(0.25),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1)),
        transforms.Lambda(lambda x: (x - mean_) / std_),
    ]
)

data_transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Lambda(lambda x: (x - mean_) / std_)]
)
train_dataset = DatasetTransformer(train_dataset, train_data_transforms)
valid_dataset = DatasetTransformer(valid_dataset, data_transforms)
test_dataset = DatasetTransformer(test_dataset, data_transforms)

In [ ]:
# 4. Prepare data loader
num_threads = 4  # Loading the dataset is using 4 CPU threads
batch_size = 512  # Using minibatches of 512 samples

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_threads
)

valid_loader = torch.utils.data.DataLoader(
    dataset=valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_threads
)


test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_threads
)

# Model definition

In [ ]:
class Conv2DClassificationModel(nn.Module):
    def __init__(self, n_classes: int, p: float = 0.5):
        super(Conv2DClassificationModel, self).__init__()
        self.n_classes = n_classes
        self.layers = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=5, padding=2),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 16, kernel_size=5, padding=2),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # nn.Conv2d(32, 64, kernel_size=5, padding=2),
            # nn.LazyBatchNorm2d(),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(128),
            nn.ReLU(),
            nn.Dropout(p),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.Dropout(p),
            nn.LazyLinear(self.n_classes),
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        y = self.layers(X)
        return y

    def predict_proba(self, X: torch.Tensor) -> torch.Tensor:
        y = self.forward(X)
        p = self.softmax(y)
        return p

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        p = self.forward(X)
        c = torch.argmax(p, dim=1)
        return c

In [ ]:
n_epochs = 50
learning_rate = 0.005
model = Conv2DClassificationModel(10)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_set_len = len(train_loader)
val_set_len = len(valid_loader)
train_loss, val_loss = [], []

for epoch in range(n_epochs):
    model.train()
    accu = 0.0

    for X_, y_ in train_loader:
        # Forward pass
        y_hat = model(X_)
        loss = loss_fn(y_hat, y_)
        accu += loss.item()

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss.append(accu / train_set_len)

    # Validation - no gradient & eval mode
    model.eval()
    accu = 0.0
    with torch.no_grad():
        for X_, y_ in valid_loader:
            # Forward pass
            y_hat = model(X_)
            loss = loss_fn(y_hat, y_)
            accu += loss.item()
        val_loss.append(accu / val_set_len)

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.grid()

In [ ]:
model.eval()
acc = 0.0
with torch.no_grad():
    for X_, y_ in test_loader:
        y_hat = model.predict(X_)
        acc += (y_ == y_hat).sum().item()
    acc /= len(test_dataset)
print(acc)

In [ ]:
# transform_image = transforms.Compose(
#     [
#         transforms.RandomHorizontalFlip(0.5),
#         transforms.RandomAffine(degrees=10, translate=(0.2, 0.2)),
#         transforms.RandomRotation(10),
#     ]
# )
# for i in range(5):
#     imt = transform_image(X_[10, ...])Cela ne me pose pas
#     plt.figure()
#     plt.imshow(imt[0, ...])
#     plt.axis("off")
#     plt.savefig(
#         f"../Course/figures/data_augment_{i}.pdf", bbox_inches="tight", pad_inches=0
#     )
#     plt.close()